In [4]:
# dependencies
import pymongo
import nibabel as nib
from os.path import join, exists, split
from dipy.tracking import utils
import os
import numpy as np
from dipy.tracking.utils import subsegment

In [5]:
# pull painter objects from a specific freesurfer ID

def get_collection(port=3001):
    from pymongo import MongoClient
    client = MongoClient("localhost", port)
    db =  client.meteor
    collection = db.subjects
    return collection, client

collection, client = get_collection(5051)

In [39]:
# this lists all the entries associated with a specific subject id
# you can use this to see what data is there. you'll need "entry_type:lst" as that has the painters
# you'll also need "entry_type:freesurfer" for

cursor = collection.find({"subject_id":"mse65"})
for item in cursor:
    print()
    print(item)
    print()


{'subject_id': 'mse65', 'entry_type': 'nifti', 'check_masks': ['mse65/nii/ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-000.nii.gz'], 'name': 'ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-000', '_id': ObjectId('57a279df142c116598dfadfe')}


{'subject_id': 'mse65', 'entry_type': 'nifti', 'check_masks': ['mse65/nii/ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-001.nii.gz'], 'name': 'ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-001', '_id': ObjectId('57a279df142c116598dfadff')}


{'subject_id': 'mse65', 'entry_type': 'nifti', 'check_masks': ['mse65/nii/ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-002.nii.gz'], 'name': 'ms534-mse65-001-SCOUTS_132Lbs_WEIGHT-002', '_id': ObjectId('57a279df142c116598dfae00')}


{'subject_id': 'mse65', 'entry_type': 'nifti', 'check_masks': ['mse65/nii/ms534-mse65-002-AX_T1_3D_IRSPGR_te_1.6.nii.gz'], 'name': 'ms534-mse65-002-AX_T1_3D_IRSPGR_te_1.6', '_id': ObjectId('57a279df142c116598dfae01')}


{'subject_id': 'mse65', 'entry_type': 'nifti', 'check_masks': ['mse65/nii/ms534-mse65-003-FRFSE-000.nii.gz'

In [6]:
cursor = collection.find({"subject_id":"mse65", "entry_type":"freesurfer"})
results = []
for item in cursor:
    results.append(item)
assert len(results) == 1, "more than one result! either modify the code or restrict your query Amit"
subject = results[0]
subject["check_masks"]

segmentation_mask = nib.load(subject["check_masks"][1])

['mse65/masks/ms534-mse65-002-AX_T1_3D_IRSPGR_te_1.6/t1.nii.gz',
 'mse65/masks/ms534-mse65-002-AX_T1_3D_IRSPGR_te_1.6/segmentation.nii.gz']

In [42]:
cursor = collection.find({"subject_id":"mse65", "entry_type":"lst"})
results = []
for item in cursor:
    results.append(item)
assert len(results) == 1, "more than one result! either modify the code or restrict your query Amit"
painter_entry = results[0]

In [45]:
len(painter_entry['painters'])
painter_entry['painters']

[{'checkedBy': 'ssacco',
  'matrix_coor': [{'old_val': 28, 'x': 159, 'y': 162, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 162, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 163, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 164, 'z': 84},
   {'old_val': 28, 'x': 159, 'y': 161, 'z': 84},
   {'old_val': 28, 'x': 159, 'y': 160, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 160, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 161, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 158, 'z': 84},
   {'old_val': 28, 'x': 160, 'y': 157, 'z': 84},
   {'old_val': 28, 'x': 159, 'y': 159, 'z': 84}],
  'offset': [],
  'paintValue': 0,
  'start_point': {'x': 667, 'y': 680},
  'uuid': 'a86f26a7-11fb-0aa7-b499-b225e3521a3a',
  'world_coor': [{'x': 27.190002441406254,
    'y': -4.417007446289052,
    'z': 12.207099914550781},
   {'x': 28.127502441406254, 'y': -4.417007446289052, 'z': 12.207099914550781},
   {'x': 28.127502441406254, 'y': -5.354507446289052, 'z': 12.207099914550781},
   {'x': 28.127502441406254, 'y': 

In [ ]:
# ABOVE: clean up function and have it take in a subject ID and return (1) painter objects and (2) paths to data
# extract data to get the segmentation masks
# NOTE: where the data lives: /data/henry7/PBR/subjects/

In [ ]:
# affine transformations
def get_papaya_aff(img):
    vs = img.header.get_zooms()
    aff = img.get_affine()
    ort = nib.orientations.io_orientation(aff)
    papaya_aff = np.zeros((4, 4))
    for i, line in enumerate(ort):
        papaya_aff[line[0],i] = vs[i]*line[1]
    papaya_aff[:, 3] = aff[:, 3]
    return papaya_aff

In [ ]:
# convert the painter object points into the correct space

def convert_to_indices(streamline, papaya_aff, aff, img):
    topoints = lambda x : np.array([[m["x"], m["y"], m["z"]] for m in x["world_coor"]])
    points_orig = topoints(streamline)
    points_nifti_space = list(utils.move_streamlines([points_orig], aff, input_space=papaya_aff))[0]
    from dipy.tracking._utils import _to_voxel_coordinates, _mapping_to_voxel
    lin_T, offset = _mapping_to_voxel(aff, None)
    idx = _to_voxel_coordinates(streamline, lin_T, offset)
    return points_nifti_space, idx

def convert_to_volume(drawing, papaya_aff, aff, img, do_subsegment = True):

    topoints = lambda x : np.array([[m["x"], m["y"], m["z"]] for m in x["world_coor"]])
    points_orig = list(map(topoints, drawing))
    if do_subsegment:
        points = list(subsegment(points_orig, 0.5))
    else:
        points = points_orig
    mask2 = utils.density_map(points, img.shape, affine=papaya_aff)
    points_nifti_space = list(utils.move_streamlines(points, aff, input_space=papaya_aff))
    mask1 = utils.density_map(points_nifti_space, img.shape, affine=aff)

    #print((mask1 == mask2).all())
    # img1 = nib.Nifti1Image(mask1)
    #print(mask1.sum(), mask2.sum())
    
    return mask1, points_nifti_space

In [ ]:
# the actual paint function (put helper functions above where I convert to the right space?)
def paintVolume(drawing, papaya_affine, aff, img, outfilepath, name, authors, suffix=""):
    import pandas as pd
    df = pd.DataFrame()

    for i,d in enumerate(drawing):
        pv = d["paintValue"]
        mask_tmp, points_nii_space = convert_to_volume([d], papaya_affine, aff, img, False)
        nii_points = points_nii_space[0]
        tmp = []
        for ni in nii_points:
            tmp.append({"x": ni[0], "y":ni[1], "z": ni[2], "val": pv})
        df = df.append(pd.DataFrame(tmp), ignore_index=True)
    df.drop_duplicates(inplace=True)
    if not exists(join(cc["output_directory"], outfilepath)):
        os.makedirs(join(cc["output_directory"],outfilepath))
        print(join(cc["output_directory"], outfilepath), "created")
    outfilename = join(outfilepath, "{}-{}{}.csv".format(name,"-".join(authors), suffix))
    #nib.Nifti1Image(mask.astype(np.float32), affine=aff).to_filename(join(cc["output_directory"], outfilename))
    df.to_csv(join(cc["output_directory"], outfilename))
    return outfilename

def create_paint_volume(output):
    from pbr.config import config as cc
    img = nib.load(join(cc["output_directory"],output["check_masks"][-1]))
    aff = img.get_affine() #affine()
    papaya_affine = get_papaya_aff(img)
    data = np.zeros(img.shape)
    outputfiles = []
    mse = output["subject_id"]
    sequence = output["name"]
    #for c in output["contours"]:
    drawing_old = output["painters"]
    drawing = []
    for p in drawing_old:
        if len(p["world_coor"]):
            drawing.append(p)

    name = output["entry_type"] #contour['name'].replace(" ", "_")
    authors = set([q["checkedBy"] for q in drawing])

    #mask, points_nifti_space = convert_to_volume(drawing, papaya_affine, aff, img, False)

    outfilepath = join(mse, "mindcontrol/{}/{}/rois".format(sequence, output["entry_type"]))

    outfilename = paintVolume(drawing, papaya_affine, aff, img, outfilepath, name, authors)
    print("painter wrote", join(cc["output_directory"], outfilename))
    outputfiles.append(outfilename)

    outfilename = paintVolume(drawing, aff, aff, img, outfilepath, name, authors, suffix="_origAff")
    print("painter wrote", join(cc["output_directory"], outfilename))
    outputfiles.append(outfilename)
    return outputfiles